<a href="https://colab.research.google.com/github/Stargardts/Abdullah-Jat.github.io/blob/main/AKD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, PatternFill
from openpyxl.utils import get_column_letter

# ------------------- 1. LOAD DATA -------------------
file_path = 'AKD (1).xlsx'  # Your uploaded file

# Read sheets
income = pd.read_excel(file_path, sheet_name='Income Statement', header=None)
balance = pd.read_excel(file_path, sheet_name='Balance Sheet', header=None)
cashflow = pd.read_excel(file_path, sheet_name='Cash Flow', header=None)
summary = pd.read_excel(file_path, sheet_name='Financial Summary', header=None)

# Years
years = [2019, 2020, 2021, 2023, 2024, 2025]

# ------------------- 2. EXTRACT HISTORICAL (ROBUST) -------------------
def find_row(df, keywords):
    for i, cell in enumerate(df.iloc[:, 0]):
        if pd.isna(cell): continue
        cell_str = str(cell).lower()
        if any(k.lower() in cell_str for k in keywords):
            values = df.iloc[i, 1:7].astype(float).values
            return dict(zip(years, values))
    print(f"Not found: {keywords}")
    return dict(zip(years, [0]*6))

hist = {
    'Net_Income': find_row(income, ['Income before Discontinued Operations & Extraordinary Items', 'Net Income']),
    'Book_Value': find_row(balance, ['Common Equity - Total', 'Total Shareholders Equity']),
    'Dividends': find_row(cashflow, ['Dividends Paid - Cash - Total']),
    'ROE': find_row(summary, ['Return on Average Common Equity', 'ROE']),
    'Shares': find_row(balance, ['Common Shares - Outstanding - Total']),
}

df_hist = pd.DataFrame(hist).T

# ------------------- 3. ASSUMPTIONS -------------------
assumptions = {
    'Forecast Years': 5,                   # Explicit forecast period (2026-2030)
    'Terminal Growth Rate': 0.05,          # Perpetual growth (Pakistan nominal GDP est.)
    'Initial ROE': df_hist.loc['ROE', 2025] / 100 if not np.isnan(df_hist.loc['ROE', 2025]) else 0.295,  # 2025 ROE
    'Terminal ROE': 0.12,                  # Sustainable long-term ROE (industry avg)
    'Payout Ratio': 0.30,                  # Explicit period (conservative retention)
    'Terminal Payout': 1 - (0.05 / 0.12),  # From g = ROE * (1 - payout)

    # CAPM for Cost of Equity (ke)
    'Risk Free Rate': 0.1189,              # Pakistan 10Y Dec 2025
    'Beta': 0.65,                          # AKDS.PSX beta Dec 2025 (from search)
    'Market Risk Premium': 0.1635,         # Pakistan ERP 2025
    'Current Share Price': 34.95,          # AKDS.PSX price Dec 9, 2025 (latest avail)
}

# Calculate ke
ke = assumptions['Risk Free Rate'] + assumptions['Beta'] * assumptions['Market Risk Premium']
assumptions['Cost of Equity'] = ke

# ROE Fade: Linear interpolation to terminal
roe_fade = np.linspace(assumptions['Initial ROE'], assumptions['Terminal ROE'], assumptions['Forecast Years'] + 1)[1:]

# ------------------- 4. FORECAST RESIDUAL INCOME -------------------
last_bv = df_hist.loc['Book_Value', 2025]
forecast_years = list(range(2026, 2031))
ri_list = []

current_bv = last_bv
for i, year in enumerate(forecast_years):
    roe = roe_fade[i]
    ni = roe * current_bv
    div = ni * assumptions['Payout Ratio']
    next_bv = current_bv + ni - div
    ri = ni - (ke * current_bv)

    ri_list.append({
        'Year': year,
        'Beg BV': current_bv,
        'ROE': roe,
        'Net Income': ni,
        'Dividends': div,
        'End BV': next_bv,
        'Residual Income': ri
    })

    current_bv = next_bv

df_forecast = pd.DataFrame(ri_list)

# PV of Explicit RI
discount_factors = [(1 + ke) ** (i+1) for i in range(5)]
pv_ri_explicit = sum(df_forecast.iloc[i]['Residual Income'] / discount_factors[i] for i in range(5))

# Terminal RI (perpetual)
terminal_ri = df_forecast.iloc[-1]['Residual Income'] * (1 + assumptions['Terminal Growth Rate'])
terminal_value = terminal_ri / (ke - assumptions['Terminal Growth Rate'])
pv_terminal = terminal_value / discount_factors[-1]

# Total Value
equity_value = last_bv + pv_ri_explicit + pv_terminal
shares = df_hist.loc['Shares', 2025]
intrinsic_price = equity_value / shares

# Recommendation
margin_of_safety = (intrinsic_price - assumptions['Current Share Price']) / assumptions['Current Share Price']
recommendation = "STRONG BUY" if margin_of_safety > 0.3 else "BUY" if margin_of_safety > 0.1 else "HOLD" if margin_of_safety > -0.1 else "SELL"

# ------------------- 5. SENSITIVITY ANALYSIS -------------------
# Vary ke ±1%, g ±1%, terminal ROE ±1% (3x3 tables)

# Helper function to compute intrinsic price with varied params
def compute_intrinsic(vary_ke=None, vary_g=None, vary_term_roe=None):
    local_ke = vary_ke if vary_ke else ke
    local_g = vary_g if vary_g else assumptions['Terminal Growth Rate']
    local_term_roe = vary_term_roe if vary_term_roe else assumptions['Terminal ROE']

    # Adjust fade
    local_roe_fade = np.linspace(assumptions['Initial ROE'], local_term_roe, assumptions['Forecast Years'] + 1)[1:]

    # Re-forecast
    curr_bv = last_bv
    local_ri = []
    for i in range(5):
        local_roe = local_roe_fade[i]
        local_ni = local_roe * curr_bv
        local_div = local_ni * assumptions['Payout Ratio']
        next_bv = curr_bv + local_ni - local_div
        local_ri.append(local_ni - (local_ke * curr_bv))
        curr_bv = next_bv

    # PV explicit
    local_discounts = [(1 + local_ke) ** (i+1) for i in range(5)]
    local_pv_explicit = sum(local_ri[i] / local_discounts[i] for i in range(5))

    # Terminal
    local_term_ri = local_ri[-1] * (1 + local_g)
    local_term_val = local_term_ri / (local_ke - local_g) if local_ke > local_g else np.inf
    local_pv_term = local_term_val / local_discounts[-1]

    local_equity = last_bv + local_pv_explicit + local_pv_term
    return local_equity / shares

# ke vs g sensitivity
ke_vars = [ke - 0.01, ke, ke + 0.01]
g_vars = [assumptions['Terminal Growth Rate'] - 0.01, assumptions['Terminal Growth Rate'], assumptions['Terminal Growth Rate'] + 0.01]
sens_ke_g = [[compute_intrinsic(vary_ke=k, vary_g=g) for g in g_vars] for k in ke_vars]
df_sens_ke_g = pd.DataFrame(sens_ke_g, index=[f'{k*100:.2f}%' for k in ke_vars], columns=[f'{g*100:.2f}%' for g in g_vars])

# ke vs term ROE
term_roe_vars = [assumptions['Terminal ROE'] - 0.01, assumptions['Terminal ROE'], assumptions['Terminal ROE'] + 0.01]
sens_ke_roe = [[compute_intrinsic(vary_ke=k, vary_term_roe=r) for r in term_roe_vars] for k in ke_vars]
df_sens_ke_roe = pd.DataFrame(sens_ke_roe, index=[f'{k*100:.2f}%' for k in ke_vars], columns=[f'{r*100:.2f}%' for r in term_roe_vars])

# ------------------- 6. CREATE EXCEL FILE -------------------
wb = Workbook()
ws = wb.active
ws.title = "RIM Valuation"

# Title
ws['A1'] = "AKD Securities Ltd (AKDS.PSX) - Residual Income Model (RIM) with Sensitivity"
ws['A1'].font = Font(size=16, bold=True)

# Assumptions
ws['A3'] = "ASSUMPTIONS"
ws['A3'].font = Font(bold=True, color="FFFFFF")
ws['A3'].fill = PatternFill(start_color="366092", end_color="366092", fill_type="solid")

row = 4
for k, v in assumptions.items():
    ws[f'A{row}'] = k
    ws[f'B{row}'] = round(v, 4) if isinstance(v, float) else v
    row += 1

# Historical
ws['D3'] = "HISTORICAL DATA (PKR mn)"
ws['D3'].font = Font(bold=True)
row = 4
for year in years:
    ws.cell(row=row, column=4 + years.index(year) + 1, value=year).font = Font(bold=True)
row += 1
for item, data in df_hist.iterrows():
    ws.cell(row=row, column=4, value=item)
    for i, year in enumerate(years):
        ws.cell(row=row, column=5+i, value=round(data[year], 2))
    row += 1

# Forecast
start_row = row + 2
ws.cell(row=start_row, column=1, value="RIM FORECAST & VALUATION").font = Font(bold=True, size=12)
ws.cell(row=start_row+1, column=1, value="Year")
for i, year in enumerate(forecast_years):
    ws.cell(row=start_row+1, column=i+2, value=year).font = Font(bold=True)

items = ['Beg BV', 'ROE', 'Net Income', 'Dividends', 'End BV', 'Residual Income']
for i, item in enumerate(items):
    ws.cell(row=start_row+2+i, column=1, value=item)
    for j in range(5):
        val = df_forecast.iloc[j][item]
        ws.cell(row=start_row+2+i, column=j+2, value=round(val, 2))

# Valuation Summary
val_row = start_row + 9
ws.cell(row=val_row, column=1, value="VALUATION SUMMARY").font = Font(bold=True, size=14)
ws.cell(row=val_row+1, column=1, value="Current Book Value").font = Font(bold=True)
ws.cell(row=val_row+1, column=2, value=round(last_bv, 2))
ws.cell(row=val_row+2, column=1, value="PV Explicit RI").font = Font(bold=True)
ws.cell(row=val_row+2, column=2, value=round(pv_ri_explicit, 2))
ws.cell(row=val_row+3, column=1, value="PV Terminal Value").font = Font(bold=True)
ws.cell(row=val_row+3, column=2, value=round(pv_terminal, 2))
ws.cell(row=val_row+4, column=1, value="Equity Value").font = Font(bold=True)
ws.cell(row=val_row+4, column=2, value=round(equity_value, 2))
ws.cell(row=val_row+5, column=1, value="Shares Outstanding (mn)").font = Font(bold=True)
ws.cell(row=val_row+5, column=2, value=shares)
ws.cell(row=val_row+6, column=1, value="Intrinsic Value per Share").font = Font(bold=True, size=14)
ws.cell(row=val_row+6, column=2, value=round(intrinsic_price, 2)).font = Font(size=14, bold=True, color="000080")
ws.cell(row=val_row+7, column=1, value="Current Market Price").font = Font(bold=True)
ws.cell(row=val_row+7, column=2, value=assumptions['Current Share Price'])
ws.cell(row=val_row+8, column=1, value="Upside/(Downside)").font = Font(bold=True)
ws.cell(row=val_row+8, column=2, value=f"{margin_of_safety:+.1%}")
ws.cell(row=val_row+9, column=1, value="RECOMMENDATION").font = Font(bold=True, size=16)
ws.cell(row=val_row+9, column=2, value=recommendation).font = Font(size=16, bold=True, color="006400" if "BUY" in recommendation else "DC143C")

# Sensitivity Analysis - ke vs g
sens_row = val_row + 11
ws.cell(row=sens_row, column=1, value="SENSITIVITY: Intrinsic Price (ke vs Terminal Growth)").font = Font(bold=True, size=12)
ws.cell(row=sens_row+1, column=1, value="ke \ g")
for i, g in enumerate(g_vars):
    ws.cell(row=sens_row+1, column=i+2, value=f"{g*100:.2f}%").font = Font(bold=True)
for j, k in enumerate(ke_vars):
    ws.cell(row=sens_row+2+j, column=1, value=f"{k*100:.2f}%").font = Font(bold=True)
    for i in range(3):
        val = df_sens_ke_g.iloc[j, i]
        ws.cell(row=sens_row+2+j, column=i+2, value=round(val, 2) if np.isfinite(val) else 'N/A')

# Sensitivity - ke vs Terminal ROE
sens_row += 6
ws.cell(row=sens_row, column=1, value="SENSITIVITY: Intrinsic Price (ke vs Terminal ROE)").font = Font(bold=True, size=12)
ws.cell(row=sens_row+1, column=1, value="ke \ Term ROE")
for i, r in enumerate(term_roe_vars):
    ws.cell(row=sens_row+1, column=i+2, value=f"{r*100:.2f}%").font = Font(bold=True)
for j, k in enumerate(ke_vars):
    ws.cell(row=sens_row+2+j, column=1, value=f"{k*100:.2f}%").font = Font(bold=True)
    for i in range(3):
        val = df_sens_ke_roe.iloc[j, i]
        ws.cell(row=sens_row+2+j, column=i+2, value=round(val, 2) if np.isfinite(val) else 'N/A')

# Save
output_file = "AKD_RIM_Valuation_Model.xlsx"
wb.save(output_file)

print(f"\nSUCCESS! RIM Model with Sensitivity created: {output_file}")
print(f"Intrinsic Value per Share: PKR {intrinsic_price:.2f}")
print(f"Current Price: PKR {assumptions['Current Share Price']:.2f}")
print(f"Recommendation: {recommendation}")
print(f"Margin of Safety: {margin_of_safety:+.1%}")

<>:226: SyntaxWarning: invalid escape sequence '\ '
<>:238: SyntaxWarning: invalid escape sequence '\ '
<>:226: SyntaxWarning: invalid escape sequence '\ '
<>:238: SyntaxWarning: invalid escape sequence '\ '
/tmp/ipython-input-168730831.py:226: SyntaxWarning: invalid escape sequence '\ '
  ws.cell(row=sens_row+1, column=1, value="ke \ g")
/tmp/ipython-input-168730831.py:238: SyntaxWarning: invalid escape sequence '\ '
  ws.cell(row=sens_row+1, column=1, value="ke \ Term ROE")



SUCCESS! RIM Model with Sensitivity created: AKD_RIM_Valuation_Model.xlsx
Intrinsic Value per Share: PKR 10.66
Current Price: PKR 34.95
Recommendation: SELL
Margin of Safety: -69.5%


In [2]:
from google.colab import files
files.download('AKD_RIM_Valuation_Model.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install yfinance pandas numpy statsmodels

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# ============================
# 1. Load Your Data
# ============================

akd = pd.read_csv("akd.csv")
kse = pd.read_csv("kse100.csv")

akd["Date"] = pd.to_datetime(akd["Date"])
kse["Date"] = pd.to_datetime(kse["Date"])

# CLEAN PRICE COLUMNS (critical!)
akd["Price"] = pd.to_numeric(
    akd["Price"].astype(str).str.replace(",", "").str.strip(),
    errors="coerce"
)

kse["Price"] = pd.to_numeric(
    kse["Price"].astype(str).str.replace(",", "").str.strip(),
    errors="coerce"
)

# Keep only required columns
akd = akd[["Date", "Price"]].rename(columns={"Price": "AKD_Price"})
kse = kse[["Date", "Price"]].rename(columns={"Price": "KSE_Price"})

# ============================
# 2. Merge the two datasets
# ============================

df = akd.merge(kse, on="Date", how="inner")
df = df.sort_values("Date").reset_index(drop=True)

# Remove rows with missing price
df = df.dropna(subset=["AKD_Price", "KSE_Price"])

# ============================
# 3. Compute Log Returns
# ============================

df["AKD_Returns"] = np.log(df["AKD_Price"] / df["AKD_Price"].shift(1))
df["KSE_Returns"] = np.log(df["KSE_Price"] / df["KSE_Price"].shift(1))

df = df.dropna()

# ============================
# 4. Regression Beta
# ============================

X = sm.add_constant(df["KSE_Returns"])
y = df["AKD_Returns"]

model = sm.OLS(y, X).fit()

regression_beta = model.params["KSE_Returns"]
alpha = model.params["const"]
r_squared = model.rsquared

# ============================
# 5. Covariance Beta
# ============================

cov = np.cov(df["AKD_Returns"], df["KSE_Returns"])[0][1]
var_mkt = np.var(df["KSE_Returns"])
cov_beta = cov / var_mkt

# ============================
# 6. Adjusted Beta
# ============================

adjusted_beta = 0.67 * regression_beta + 0.33 * 1.0

# ============================
# 7. Output
# ============================

print("====== BETA RESULTS ======")
print(f"Regression Beta: {regression_beta:.4f}")
print(f"Covariance Beta: {cov_beta:.4f}")
print(f"Adjusted Beta: {adjusted_beta:.4f}")
print(f"Alpha: {alpha:.6f}")
print(f"R-Squared: {r_squared:.4f}")

df.to_csv("beta_dataset_output.csv", index=False)
print("\nSaved: beta_dataset_output.csv")


====== BETA RESULTS ======
Regression Beta: 0.8233
Covariance Beta: 0.8375
Adjusted Beta: 0.8816
Alpha: -0.006518
R-Squared: 0.0876

Saved: beta_dataset_output.csv
